In [ ]:
# ============================================
# Install dependencies
# ============================================
%pip install optuna --quiet
%pip install torch torchvision torchaudio --quiet

# ============================================
# Imports
# ============================================
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import random
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    precision_recall_fscore_support,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)
from sklearn.utils import resample

import optuna

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

# ============================================
# Global seed & deterministic setup
# ============================================
SEED = 42

random.seed(SEED)
np.random.seed(SEED)

torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# cuDNN deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Try strict deterministic algorithms (may raise on some ops/versions)
try:
    torch.use_deterministic_algorithms(True)
    print("Using strict deterministic algorithms in PyTorch.")
except Exception as e:
    print("Warning: Could not enable strict deterministic algorithms:", e)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================
# Load dataset and feature selection
# ============================================
features = [
    'RiskScore', 'MonthlyCharges', 'AvgChargesPerMonth', 'ContractRisk', 'Contract',
    'TotalServices', 'TotalCharges', 'EstimatedCLV', 'HasFamily', 'tenure',
    'Partner', 'HighValueCustomer', 'PaymentRisk', 'Dependents'
]

df = pd.read_csv('final_telco_engineered.csv')

X = df[features]
y = df['Churn']  # adjust if your target column name differs

# Identify categorical and numerical columns for preprocessing
categorical_cols = ['Contract', 'HasFamily', 'Partner', 'HighValueCustomer', 'Dependents']
numerical_cols = [col for col in features if col not in categorical_cols]

# ============================================
# Preprocessing pipeline
# ============================================
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(drop='first'), categorical_cols)
])

# ============================================
# Train/validation/test split (80/10/10)
# ============================================
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.10, random_state=SEED, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.1111, random_state=SEED, stratify=y_temp
)  # 0.1111 * 0.90 ≈ 0.10

# Ensure target variable numeric (0/1)
if y_train.dtype == 'O' or y_train.dtype.name == 'category':
    mapping = {'No': 0, 'Yes': 1}
    y_train = y_train.map(mapping).astype(int)
    y_val   = y_val.map(mapping).astype(int)
    y_test  = y_test.map(mapping).astype(int)

# ============================================
# Fit preprocessor on ORIGINAL training data
# ============================================
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc   = preprocessor.transform(X_val)
X_test_proc  = preprocessor.transform(X_test)

# ============================================
# Random oversampling to balance classes (50/50)
# ============================================
print("Before oversampling:")
print(pd.Series(y_train).value_counts())
print(pd.Series(y_train).value_counts(normalize=True))

_df_train = pd.DataFrame(X_train_proc)
_df_train['Churn'] = y_train.reset_index(drop=True)

_df_majority = _df_train[_df_train['Churn'] == 0]
_df_minority = _df_train[_df_train['Churn'] == 1]

_df_minority_up = resample(
    _df_minority,
    replace=True,
    n_samples=len(_df_majority),
    random_state=SEED,    # deterministic upsampling
)

_df_balanced = pd.concat([_df_majority, _df_minority_up], axis=0)
_df_balanced = _df_balanced.sample(frac=1, random_state=SEED).reset_index(drop=True)

X_train_proc = _df_balanced.drop(columns=['Churn']).values
y_train = _df_balanced['Churn'].astype(int).values

print("\nAfter oversampling:")
print(pd.Series(y_train).value_counts())
print(pd.Series(y_train).value_counts(normalize=True))

# ============================================
# PyTorch Dataset / DataLoaders (deterministic)
# ============================================
class TabDataset(Dataset):
    def __init__(self, X, y):
        # Handle pandas DataFrame / Series or numpy arrays
        if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values

        self.X = torch.tensor(np.asarray(X), dtype=torch.float32)
        self.y = torch.tensor(np.asarray(y), dtype=torch.long)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_dataset = TabDataset(X_train_proc, y_train)
val_dataset   = TabDataset(X_val_proc,   y_val)
test_dataset  = TabDataset(X_test_proc,  y_test)

BATCH_SIZE = 128

# Deterministic data loader generator
g = torch.Generator()
g.manual_seed(SEED)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    generator=g,
    num_workers=0,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
)

n_features = X_train_proc.shape[1]
n_classes  = 2  # 0 = No, 1 = Yes

# ============================================
# TabTransformer-like model (features as tokens)
# ============================================
class TabTransformer(nn.Module):
    def __init__(
        self,
        n_features: int,
        d_model: int = 64,
        n_heads: int = 4,
        n_layers: int = 2,
        dim_feedforward: int = 128,
        dropout: float = 0.1,
        n_classes: int = 2,
    ):
        super().__init__()
        self.n_features = n_features
        self.d_model = d_model

        # One embedding vector per feature index
        self.feature_emb = nn.Embedding(n_features, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.norm = nn.LayerNorm(d_model)
        self.cls_head = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, n_classes),
        )

    def forward(self, x):
        """
        x: (batch_size, n_features) - already standardized / one-hot encoded
        We treat each feature as a token. For feature i, representation is:
        token_i = value_i * embedding(feature_index=i)
        """
        batch_size = x.size(0)

        feat_indices = torch.arange(self.n_features, device=x.device)
        feat_emb = self.feature_emb(feat_indices)        # (n_features, d_model)
        feat_emb = feat_emb.unsqueeze(0).expand(batch_size, -1, -1)  # (B, n_features, d_model)

        x_expanded = x.unsqueeze(-1)                     # (B, n_features, 1)
        tokens = feat_emb * x_expanded                   # (B, n_features, d_model)

        h = self.transformer(tokens)                     # (B, n_features, d_model)
        h = self.norm(h)

        h_pool = h.mean(dim=1)                           # (B, d_model)
        logits = self.cls_head(h_pool)                   # (B, n_classes)
        return logits

# ============================================
# Training / evaluation helpers
# ============================================
from sklearn.metrics import f1_score

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    all_preds, all_targets = [], []

    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds.detach().cpu().numpy())
        all_targets.append(y_batch.detach().cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)
    avg_loss = running_loss / len(loader.dataset)
    f1 = f1_score(all_targets, all_preds, pos_label=1, zero_division=0)
    return avg_loss, f1


def eval_model(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    all_preds, all_targets = [], []
    all_probs = []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            running_loss += loss.item() * X_batch.size(0)
            probs = torch.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=1)

            all_probs.append(probs.cpu().numpy())
            all_preds.append(preds.cpu().numpy())
            all_targets.append(y_batch.cpu().numpy())

    all_probs = np.concatenate(all_probs)
    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    avg_loss = running_loss / len(loader.dataset)
    f1 = f1_score(all_targets, all_preds, pos_label=1, zero_division=0)
    return avg_loss, f1, all_targets, all_preds, all_probs

# ============================================
# Optuna objective for TabTransformer (deterministic)
# ============================================
PATIENCE = 5
MAX_EPOCHS = 50

# Replace F1-based early stopping with ROC AUC monitoring
def objective_tabtransformer(trial):
    d_model = trial.suggest_categorical("d_model", [32, 64, 128])
    n_heads = trial.suggest_categorical("n_heads", [2, 4])
    n_layers = trial.suggest_int("n_layers", 1, 4)
    dim_feedforward = trial.suggest_categorical("dim_feedforward", [64, 128, 256])
    dropout = trial.suggest_float("dropout", 0.0, 0.3)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    model = TabTransformer(
        n_features=n_features,
        d_model=d_model,
        n_heads=n_heads,
        n_layers=n_layers,
        dim_feedforward=dim_feedforward,
        dropout=dropout,
        n_classes=n_classes,
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    best_val_auc = 0.0
    epochs_no_improve = 0

    for epoch in range(MAX_EPOCHS):
        train_loss, train_f1 = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_f1, y_val_true, _, y_val_prob = eval_model(model, val_loader, criterion)
        val_auc = roc_auc_score(y_val_true, y_val_prob)

        trial.report(val_auc, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

        if val_auc > best_val_auc + 1e-5:
            best_val_auc = val_auc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= PATIENCE:
            print(f"[Trial {trial.number}] early stopping at epoch {epoch+1} (best val AUC={best_val_auc:.4f})")
            break

    return best_val_auc

# Run Optuna study
study_tab = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    study_name="tabtransformer_fixed",
)
study_tab.optimize(objective_tabtransformer, n_trials=100)

print("Best trial:", study_tab.best_trial.number)
print("Best params:", study_tab.best_params)
print("Best val AUC:", study_tab.best_value)

# Train final model with best hyperparameters using val AUC early stopping
best_params = study_tab.best_params
best_model = TabTransformer(
    n_features=n_features,
    d_model=best_params["d_model"],
    n_heads=best_params["n_heads"],
    n_layers=best_params["n_layers"],
    dim_feedforward=best_params["dim_feedforward"],
    dropout=best_params["dropout"],
    n_classes=n_classes,
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(best_model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])

best_val_auc = 0.0
epochs_no_improve = 0
for epoch in range(MAX_EPOCHS):
    train_loss, train_f1 = train_one_epoch(best_model, train_loader, optimizer, criterion)
    val_loss, val_f1, y_val_true, _, y_val_prob = eval_model(best_model, val_loader, criterion)
    val_auc = roc_auc_score(y_val_true, y_val_prob)
    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val AUC: {val_auc:.4f}")

    if val_auc > best_val_auc + 1e-5:
        best_val_auc = val_auc
        best_state_dict = best_model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= PATIENCE:
        print(f"Early stopping triggered (best val AUC={best_val_auc:.4f}).")
        break

best_model.load_state_dict(best_state_dict)

# ============================================
# Evaluate on test set
# ============================================
test_loss, test_f1, y_true, y_pred, y_proba = eval_model(best_model, test_loader, criterion)

accuracy = (y_true == y_pred).mean()
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true, y_pred, pos_label=1, average="binary", zero_division=0
)

cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

auc = roc_auc_score(y_true, y_proba)

print("\n=== TabTransformer Test Metrics ===")
print(f"Accuracy   : {accuracy:.4f}")
print(f"Precision  : {precision:.4f}")
print(f"Recall     : {recall:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F1 score   : {f1:.4f}")
print(f"AUC        : {auc:.4f}")

print("\nConfusion Matrix:")
print(cm)

# ============================================
# Plot Confusion Matrix
# ============================================
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("TabTransformer Confusion Matrix (Test)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# ============================================
# Plot ROC Curve
# ============================================
fpr, tpr, _ = roc_curve(y_true, y_proba)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("TabTransformer ROC Curve (Test)")
plt.legend()
plt.show()



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/Users/shreyasshah/Desktop/Fall_2025/STA 221/ecs221/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-21 14:26:55,661] A new study created in memory with name: tabtransformer_fixed


Using strict deterministic algorithms in PyTorch.
Using device: cpu
Before oversampling:
Churn
0    4138
1    1495
Name: count, dtype: int64
Churn
0    0.7346
1    0.2654
Name: proportion, dtype: float64

After oversampling:
0    4138
1    4138
Name: count, dtype: int64
0    0.5
1    0.5
Name: proportion, dtype: float64


[I 2025-11-21 14:27:44,381] Trial 0 finished with value: 0.8474284062519357 and parameters: {'d_model': 64, 'n_heads': 2, 'n_layers': 1, 'dim_feedforward': 128, 'dropout': 0.21242177333881365, 'lr': 0.00010994335574766199, 'weight_decay': 0.0008123245085588687}. Best is trial 0 with value: 0.8474284062519357.


[Trial 0] early stopping at epoch 38 (best val AUC=0.8474)


...

[I 2025-11-21 14:29:47,288] Trial 3 finished with value: 0.8467057584704645 and parameters: {'d_model': 128, 'n_heads': 2, 'n_layers': 2, 'dim_feedforward': 128, 'dropout': 0.19875668530619459, 'lr': 0.0004201672054372534, 'weight_decay': 3.632486956676606e-05}. Best is trial 1 with value: 0.8490492019903785.


[Trial 3] early stopping at epoch 13 (best val AUC=0.8467)


[I 2025-11-21 14:31:38,033] Trial 4 finished with value: 0.8499267028678793 and parameters: {'d_model': 128, 'n_heads': 4, 'n_layers': 4, 'dim_feedforward': 128, 'dropout': 0.058794858725743554, 'lr': 0.00012315571723666037, 'weight_decay': 9.462175356461487e-06}. Best is trial 4 with value: 0.8499267028678793.


[Trial 4] early stopping at epoch 12 (best val AUC=0.8499)


[I 2025-11-21 14:31:43,238] Trial 5 pruned. 
[I 2025-11-21 14:31:45,107] Trial 6 pruned. 
[I 2025-11-21 14:32:00,852] Trial 7 pruned. 
[I 2025-11-21 14:32:05,600] Trial 8 pruned. 
[I 2025-11-21 14:32:24,545] Trial 9 pruned. 
